In [1]:
import scipy.io as sio    
import numpy as np
import pandas as pd
import glob,os
from matplotlib import pylab as plt
import CFS
import mrmr

In [2]:
# load data
path = '/Users/lyuliangyi/Desktop/HKCOCHE/feature_selection/All_features'
file = glob.glob(os.path.join(path,'*.mat'))

label = []
for f in file:
    label.append(f[65:-4])
    
d={}
for i in range(len(file)):
    mat = sio.loadmat(file[i])
    d['mat_'+label[i]] = pd.DataFrame(mat['feature_person'],
                                     columns=['PTTp','PTTf','upTime','BfTime','DownTime',
                                              'FbTime','BbTime','CfSlope','BcSlope','CbSlope',
                                              'sys_vol','sys_notch_vol','notch_dia_vol','dia_vol','sys_dia_RI',
                                              'sys_foot_RI', 'sys_notch_RI', 'dia_notch_RI', 'HR', 'PSR',
                                             'PSR1', 'PSR2', 'PSR3', 'PSR4', 'PSR5',
                                             'PSR6', 'PSR7', 'PSR8', 'SBP', 'DBP',
                                             'MBP'])
    
# classification of bp
for l in label:
    df = d['mat_'+l]
    df['class'] = ''
    for i in range(len(df)):
        if df.loc[i,'SBP'] >= 160 or df.loc[i,'DBP'] >= 100: # stage 1 hypertension 
            df.loc[i,'class'] = 'stage 2' 
        elif df.loc[i,'SBP'] >= 140 or df.loc[i,'DBP'] >= 90: # stage 1 hypertension 
            df.loc[i,'class'] = 'stage 1'
        elif df.loc[i,'SBP'] >= 120 or df.loc[i,'DBP'] >= 90: # prehypertension
            df.loc[i,'class'] = 'prehypertension'
        else:
            df.loc[i,'class'] = 'normal'
    d['mat_'+l] = df

# random select 50% data
d1 = []
sample_rate = 0.5

for l in label:
    m, n = d['mat_'+l].shape
    df = d['mat_'+l].sample(n=int(m*sample_rate),random_state=2023)
    d1.append(df)
df_sample = pd.concat(d1)
df_sample = df_sample.reset_index(drop=True)
df_sample = df_sample[['PTTp','PTTf','upTime','BfTime','DownTime',
                        'FbTime','BbTime','CfSlope','BcSlope','CbSlope',
                        'sys_vol','sys_notch_vol','notch_dia_vol','dia_vol','sys_dia_RI',
                        'sys_foot_RI', 'sys_notch_RI', 'dia_notch_RI', 'HR', 'PSR',
                        'PSR1', 'PSR2', 'PSR3', 'PSR4', 'PSR5',
                        'PSR6', 'PSR7', 'PSR8','SBP', 'DBP', 'MBP','class']]

# plt.hist(df_sample['class'])
# plt.show()    

In [3]:
# normalization
df_sample = df_sample[['PTTp','PTTf','upTime','BfTime','DownTime',
                        'FbTime','BbTime','CfSlope','BcSlope','CbSlope',
                        'sys_vol','sys_notch_vol','notch_dia_vol','dia_vol','sys_dia_RI',
                        'sys_foot_RI', 'sys_notch_RI', 'dia_notch_RI', 'HR', 'PSR',
                        'PSR1', 'PSR2', 'PSR3', 'PSR4', 'PSR5',
                        'PSR6', 'PSR7', 'PSR8','class']]

df_sample1 = df_sample.drop('class', axis=1)
df_norm = (df_sample1 - df_sample1.min())/(df_sample1.max() - df_sample1.min())
df_norm['class'] = df_sample['class']
df_norm

,PTTp,PTTf,upTime,BfTime,DownTime,FbTime,BbTime,CfSlope,BcSlope,CbSlope,...,PSR,PSR1,PSR2,PSR3,PSR4,PSR5,PSR6,PSR7,PSR8,class
0,0.315126,0.333333,0.210526,0.274074,0.202703,0.166889,0.112202,0.027969,0.047128,0.163040,...,0.275904,0.166642,0.174393,0.086185,0.168905,0.507176,0.232035,0.073103,0.105885,normal
1,0.319328,0.337662,0.210526,0.274074,0.202703,0.166889,0.112202,0.030230,0.046400,0.163378,...,0.267777,0.173625,0.189596,0.089769,0.167461,0.533416,0.233552,0.077152,0.077152,normal
2,0.306723,0.329004,0.200000,0.266667,0.209459,0.166889,0.110799,0.026324,0.046214,0.145890,...,0.256806,0.183752,0.183752,0.092421,0.152378,0.559973,0.256738,0.083025,0.083025,normal
3,0.302521,0.329004,0.189474,0.274074,0.216216,0.165554,0.110799,0.034745,0.050346,0.137627,...,0.247460,0.193087,0.193087,0.097117,0.162071,0.553375,0.283572,0.088438,0.088438,normal
4,0.315126,0.337662,0.200000,0.259259,0.209459,0.168224,0.110799,0.034008,0.048581,0.152102,...,0.264425,0.176630,0.183311,0.090252,0.144982,0.540990,0.266330,0.078895,0.078895,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551650,0.298319,0.324675,0.189474,0.340741,0.209459,0.148198,0.105189,0.006136,0.033578,0.162148,...,0.182141,0.285059,0.319253,0.399697,0.257579,0.426875,0.317468,0.141774,0.141774,normal
551651,0.382353,0.402597,0.210526,0.333333,0.256757,0.150868,0.106592,0.000787,0.022479,0.086691,...,0.145266,0.373482,0.389478,0.478343,0.349401,0.493752,0.341734,0.193051,0.193051,normal
551652,0.373950,0.398268,0.200000,0.355556,0.229730,0.145527,0.105189,0.004633,0.031061,0.140787,...,0.179500,0.290185,0.325400,0.418529,0.262902,0.448269,0.271560,0.144746,0.144746,normal
551653,0.378151,0.398268,0.210526,0.325926,0.277027,0.150868,0.105189,0.007498,0.027418,0.098681,...,0.178688,0.291793,0.329769,0.436253,0.264571,0.428042,0.285005,0.145679,0.145679,normal


In [4]:
# CFS
X =df_norm[['PTTp','PTTf','upTime','BfTime','DownTime',
                        'FbTime','BbTime','CfSlope','BcSlope','CbSlope',
                        'sys_vol','sys_notch_vol','notch_dia_vol','dia_vol','sys_dia_RI',
                        'sys_foot_RI', 'sys_notch_RI', 'dia_notch_RI', 'HR', 'PSR',
                        'PSR1', 'PSR2', 'PSR3', 'PSR4', 'PSR5',
                        'PSR6', 'PSR7', 'PSR8']].to_numpy()
X = X.astype(float)
y =df_norm['class'].to_numpy()
n_samples, n_features = X.shape 

CFS.cfs(X, y)

[0]
0.014269718965907292
0.014269718965907292
[1]
0.015290641686755273
0.015290641686755273
[2]
0.012217118476114713
[3]
0.008008410350478648
[4]
0.015394763244417606
0.015394763244417606
[5]
0.011524764597193066
[6]
0.009495816654999857
[7]
0.1736761510639115
0.1736761510639115
[8]
0.17366694923726964
[9]
0.17368603308176306
0.17368603308176306
[10]
0.17371772813677241
0.17371772813677241
[11]
0.17372276115659657
0.17372276115659657
[12]
0.17372077333577937
[13]
0.17372415394056168
0.17372415394056168
[14]
0.17372507622177735
0.17372507622177735
[15]
0.17372489093393373
[16]
0.17372490973621774
[17]
0.17372198686214088
[18]
0.010509164576939133
[19]
0.006174919001377813
[20]
0.006189873091833301
[21]
0.07379186570743855
[22]
0.0839015632777958
[23]
0.08316123012634981
[24]
0.1407929614629988
[25]
0.10951332651312895
[26]
0.08810674756977095
[27]
0.08509443886719777
[14, 0]
0.119148093168728
0.119148093168728
[14, 1]
0.11995753051245166
0.11995753051245166
[14, 2]
0.12300667054726008
0

0.20896576065368747
[14, 16, 15, 13, 11, 17, 12, 21]
0.2175961363605331
[14, 16, 15, 13, 11, 17, 12, 22]
0.2192068494597741
[14, 16, 15, 13, 11, 17, 12, 23]
0.21866431457364482
[14, 16, 15, 13, 11, 17, 12, 24]
0.2265208241476538
[14, 16, 15, 13, 11, 17, 12, 25]
0.2216841620122107
[14, 16, 15, 13, 11, 17, 12, 26]
0.2198175243062962
[14, 16, 15, 13, 11, 17, 12, 27]
0.21920894334238175
[14, 16, 15, 13, 11, 17, 12, 10, 0]
0.2195009756551572
0.2195009756551572
[14, 16, 15, 13, 11, 17, 12, 10, 1]
0.21980537464355265
0.21980537464355265
[14, 16, 15, 13, 11, 17, 12, 10, 2]
0.22369713460825122
0.22369713460825122
[14, 16, 15, 13, 11, 17, 12, 10, 3]
0.22137149923422708
[14, 16, 15, 13, 11, 17, 12, 10, 4]
0.22170233572856052
[14, 16, 15, 13, 11, 17, 12, 10, 5]
0.2197951013020368
[14, 16, 15, 13, 11, 17, 12, 10, 6]
0.21929969766872057
[14, 16, 15, 13, 11, 17, 12, 10, 7]
0.23306776544011262
0.23306776544011262
[14, 16, 15, 13, 11, 17, 12, 10, 8]
0.23306659976207444
[14, 16, 15, 13, 11, 17, 12, 10, 

0.21199295441658633
[14, 16, 15, 13, 11, 17, 12, 10, 9, 7, 8, 24, 2, 25, 20]
0.21199412576930046
[14, 16, 15, 13, 11, 17, 12, 10, 9, 7, 8, 24, 2, 25, 21]
0.21673503366817742
[14, 16, 15, 13, 11, 17, 12, 10, 9, 7, 8, 24, 2, 25, 22]
0.21763876505821247
[14, 16, 15, 13, 11, 17, 12, 10, 9, 7, 8, 24, 2, 25, 23]
0.21732218017008773
[14, 16, 15, 13, 11, 17, 12, 10, 9, 7, 8, 24, 2, 25, 26]
0.2179812593058162
0.2179812593058162
[14, 16, 15, 13, 11, 17, 12, 10, 9, 7, 8, 24, 2, 25, 27]
0.21763551085026656


(array([14, 16, 15, 13, 11, 17, 12, 10,  9,  7,  8, 24,  2, 25, 26]),
 [0.17372507622177735,
  0.2006004320361956,
  0.21276889895227918,
  0.21974655862249728,
  0.2242774799347869,
  0.2274584923786186,
  0.22981513432776093,
  0.23163096602969468,
  0.2330691442622227,
  0.23423804083705363,
  0.23520674382983528,
  0.23258140426035168,
  0.22764362482149145,
  0.22300863903951404,
  0.2179812593058162])